### Using Langchain to connect to Elasticserach.

We will be using Elasticsearch as our vectorDB.

In [2]:
import getpass

from langchain_elasticsearch import ElasticsearchStore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import OllamaEmbeddings

ELASTIC_CLOUD_ID = getpass.getpass("Enter Cloud ID")
ELASTIC_API_KEY = getpass.getpass("Enter Cloud API key")

embeddings = OllamaEmbeddings(model="mistral")
vector_store = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="workplace_index",
    embedding=embeddings,
)

### Using LangChain and Ollama so we can run Mistral locally.

In [ ]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler                                  
llm = Ollama(model="mistral", 
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))
llm("Tell me about the history of Mistral AI")

### Using Mistral to generate dense vectors

In [5]:
from langchain.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mistral")

llm_open = Ollama(  model="mistral",
                    #model='Llama2',
                    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

### Using Mistral to generate dense vectors and load them into Elasticsearch

In [ ]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["name"] = record.get("name")
    metadata["summary"] = record.get("summary")
    metadata["url"] = record.get("url")
    metadata["category"] = record.get("category")
    metadata["updated_at"] = record.get("updated_at")

    return metadata


loader = JSONLoader(
    file_path="/data/data.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=256
)
docs = loader.load_and_split(text_splitter=text_splitter)

documents = vector_store.from_documents(
    docs,
    embeddings,
    index_name="workplace_index",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

### Perform semantic search using Mistral dense vectors.

This is the classic RAG architecture. We are search through our internal documents that have been converted to dense vectors and stored in the vectorDB Elasticsearch.

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever()

qa_chain = RetrievalQA.from_chain_type(llm=llm_open,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

query = "what is the remote work policy?"
llm_response = qa_chain(query)
print("query: " + llm_response['query'])
print("result: " + llm_response['result'])
print("source document: " , llm_response['source_documents'])